In [ ]:
# Explore Altair package and hopefully produce some interactive charts to play with.

!pip install altair
!pip install vega
!pip install vega_datasets
!pip install altair_viewer

import pandas as pd
import numpy as np
import plotly as plt

import datascience
from datascience import *

from vega_datasets import data
import vega


In [ ]:
import altair as alt

In [ ]:
# Load Kenya MTF Data 

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("kenya_mtf2_mini.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
kenya_mtf2_mini = pd.concat(mtf_list, axis=0)
kenya_mtf2_mini = pd.DataFrame(data = kenya_mtf2_mini)

In [ ]:
# Check left join (should be an anti-join). 
# Need to check for c_c_27b and other missing variables

kenya_mtf2_mini

In [ ]:
# AIP Mockups

# Energy Access X
# Grid HH Detail X
# Solar based device detail (  )
# Stopgap HH detail - Nick
# Cooking Fuel Detail X
# Primary Source info X

In [ ]:
# Sample pulled from documentation 

movies = alt.UrlData(
    data.movies.url,
    format=alt.DataFormat(parse={"Release_Date":"date"})
)
ratings = ['G', 'NC-17', 'PG', 'PG-13', 'R']
genres = ['Action', 'Adventure', 'Black Comedy', 'Comedy',
       'Concert/Performance', 'Documentary', 'Drama', 'Horror', 'Musical',
       'Romantic Comedy', 'Thriller/Suspense', 'Western']

base = alt.Chart(movies, width=200, height=200).mark_point(filled=True).transform_calculate(
    Rounded_IMDB_Rating = "floor(datum.IMDB_Rating)",
    Hundred_Million_Production =  "datum.Production_Budget > 100000000.0 ? 100 : 10",
    Release_Year = "year(datum.Release_Date)"
).transform_filter(
    alt.datum.IMDB_Rating > 0
).transform_filter(
    alt.FieldOneOfPredicate(field='MPAA_Rating', oneOf=ratings)
).encode(
    x=alt.X('Worldwide_Gross:Q', scale=alt.Scale(domain=(100000,10**9), clamp=True)),
    y='IMDB_Rating:Q',
    tooltip="Title:N"
)

# A slider filter
year_slider = alt.binding_range(min=1969, max=2018, step=1)
slider_selection = alt.selection_single(bind=year_slider, fields=['Release_Year'], name="Release Year_")


filter_year = base.add_selection(
    slider_selection
).transform_filter(
    slider_selection
).properties(title="Slider Filtering")

# A dropdown filter
genre_dropdown = alt.binding_select(options=genres)
genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")

filter_genres = base.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="Dropdown Filtering")

#color changing marks
rating_radio = alt.binding_radio(options=ratings)

rating_select = alt.selection_single(fields=['MPAA_Rating'], bind=rating_radio, name="Rating")
rating_color_condition = alt.condition(rating_select,
                      alt.Color('MPAA_Rating:N', legend=None),
                      alt.value('lightgray'))

highlight_ratings = base.add_selection(
    rating_select
).encode(
    color=rating_color_condition
).properties(title="Radio Button Highlighting")

# Boolean selection for format changes
input_checkbox = alt.binding_checkbox()
checkbox_selection = alt.selection_single(bind=input_checkbox, name="Big Budget Films")

size_checkbox_condition = alt.condition(checkbox_selection,
                                        alt.SizeValue(25),
                                        alt.Size('Hundred_Million_Production:Q')
                                       )

budget_sizing = base.add_selection(
    checkbox_selection
).encode(
    size=size_checkbox_condition
).properties(title="Checkbox Formatting")

( filter_year | filter_genres) &  (highlight_ratings | budget_sizing  )

In [ ]:
# Energy Access Summary

# Variables: elc_aggr_tier, c_c_159, primary_1st_fuel, primary_stove_type

click = alt.selection_multi(encodings = ['color'])

energy_access = alt.Chart(kenya_mtf2_mini[kenya_mtf2_mini[['c_c_159']].notnull().all(1)]).mark_bar().encode(
    x = alt.X('elc_aggr_tier',
              sort=alt.EncodingSortField(field="c_c_159", op="count", order='descending'),
              axis = alt.Axis(title = "Tier")),
    y = 'count(elc_aggr_tier)',
    color = 'c_c_159',
    tooltip = 'c_c_159'
).transform_filter(click).properties(width = 400, height = 400).interactive()

# Primary fuel dropdown

fuel_primary = ['Biogas', 'Biomass Briquette', 'Charcoal', 'Coal/lignite', 
                'Crop Residue/Plant Biomass', 'Electricity', 'Firewood', 'Kerosene', 
                'LPG/cooking gas', 'Not Applicable', 'Processed biomass (pellets)/woodchips1']
fuel_dropdown=alt.binding_select(options=fuel_primary)
fuel_select=alt.selection_single(fields=['Primary_1st_fuel'],
                                bind=fuel_dropdown, name=" ")

# Filter primary 1st

filter_fuel=energy_access.add_selection(
    fuel_select 
).transform_filter(
    fuel_select
).properties(title="Tiers & Sources of Fuel")

# Interactive Legend - Locality

locality_ur = alt.Chart(kenya_mtf2_mini[kenya_mtf2_mini[['c_c_159']].notnull().all(1)]).mark_square().encode(
    y = alt.Y('locality_ur', axis = alt.Axis(title = "")),
    color = alt.condition(click, 'locality_ur', 
                          alt.value('lightgray'), 
                          legend = None)
).properties(width = 50, height = 50, selection = click)

# interactive legend - primary stove type. Need to get rid of "null"

stove_type = alt.Chart(kenya_mtf2_mini[kenya_mtf2_mini[['c_c_159']].notnull().all(1)]).mark_square().encode(
    y = alt.Y('Primary_stove_type', axis = alt.Axis(title = "")),
    color = alt.condition(click, 'Primary_stove_type', 
                          alt.value('lightgray'), 
                          legend = None)
).properties(width = 50, height = 75, selection = click)

filter_fuel | locality_ur & stove_type

In [ ]:
click = alt.selection_multi(encodings = ['color'])

#tiers = alt.Chart(kenya_mtf2_filtered).mark_bar().encode(
#    x = alt.X('elc_aggr_tier', axis = alt.Axis(title = "Tier")),
#    y = 'count(elc_aggr_tier)',
#    color = 'c_c_159',
#    tooltip = 'elc_aggr_tier'
#).properties(width = 600, height = 600, selection = click).interactive()

lighting_source = alt.Chart(kenya_mtf2_filtered).mark_bar(color='green').encode(
    x = alt.X('c_c_159', axis = alt.Axis(title='Main Source of Electricity')),
    y = 'count(c_c_159)',
    color = 'elc_aggr_tier',
    tooltip = 'count(elc_aggr_tier)'
).properties(width = 800, height = 600, selection = click).transform_filter(click).interactive()

lighting_source

In [ ]:
click = alt.selection_multi(encodings = ['color'])

tiers = alt.Chart(kenya_mtf2_filtered).mark_bar().encode(
    x = alt.X('elc_aggr_tier', axis = alt.Axis(title = "Tier")),
    y = 'count(elc_aggr_tier)',
    #scale = alt.Scale(domain = [0, 2100])),
    color = 'c_c_159',
    tooltip = 'c_c_159'
).transform_filter(click).properties(width = 750, height = 600).interactive()

hours_slider = alt.binding_range(min=0, max=24, step=1)
slider_selection = alt.selection_single(bind=hours_slider, 
                                        fields=['c_c_27b'], 
                                        name="Hours of Electricity Used per Day")
filter_hours = tiers.add_selection(
    slider_selection
).transform_filter(
    slider_selection
).properties(title="Tier/Locality Count & Breakdown of Electricity Sources")

tier_scatter = alt.Chart(kenya_mtf2_filtered).mark_point().encode(
    y = alt.Y('locality_ur', axis = alt.Axis(title = "")),
    color = alt.condition(click, 'locality_ur', 
                          alt.value('lightgray'), 
                          #alt.Color('locality_ur',
                          #         scale = alt.Scale(
                          #         domain = ['Urban', 'Rural'],
                          #         range = ['blue','green'])), 
                          legend = None)
).properties(width = 50, height = 100, selection = click)

filter_hours | tier_scatter

In [ ]:
click = alt.selection_multi(encodings = ['color'])

bins = alt.Chart(kenya_mtf2_filtered).mark_bar().encode(
    x = alt.X('c_c_27b', axis = alt.Axis(title = ("Average Hours Used from Grid (Binned)")), bin=True),
    y = 'count()',
    color = alt.Color('elc_aggr_tier', legend = None),
    tooltip = 'elc_aggr_tier'
).transform_filter(click).properties(width = 750, height = 600).interactive()

locality = alt.Chart(kenya_mtf2_filtered).mark_bar().encode(
    y = alt.Y('locality_ur', axis = alt.Axis(title = "National Locality")),
    x = 'c_c_159',
    color = 'count()',
    tooltip = 'count()'
).properties(width = 750, height = 150, selection = click)

bins & locality